# Men from Mars, Women from Venus: both can code .NET in Jupyter

@rondagdag

### .NET Interactive https://github.com/dotnet/interactive

### To install in raspberry pi
```
curl -L https://raw.githubusercontent.com/colombod/pi-top/master/setup-device.sh | bash -e
source .jupyter_venv/bin/activate
jupyter lab --no-browser

```

Rasperry Pi Sense Hat installation instructions
https://www.raspberrypi.org/documentation/hardware/sense-hat/

In [1]:
#!lsmagic

The below script needs to be able to find the current output cell; this is an easy method to get it.

.NET kernel #!csharp , #!c# , #!C# Run the code that follows using the csharp kernel. #!fsharp , #!f# , #!F# Run the code that follows using the fsharp kernel. #!pwsh , #!powershell Run the code that follows using the pwsh kernel. #!javascript , #!js Run the code that follows using the javascript kernel. #!html Run the code that follows using the html kernel. #!value Stores a value #!lsmagic List the available magic commands / directives #!markdown Convert the code that follows from Markdown into HTML #!time Time the execution of the following code in the submission. #!log Enables session logging. #!about Show version and build information #!connect Connects additional subkernels named-pipe Connects to a kernel using named pipes signalr Connects to a kernel using SignalR

csharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

fsharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

pwsh kernel #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

javascript kernel #!lsmagic List the available magic commands / directives

html kernel #!lsmagic List the available magic commands / directives

value kernel #!lsmagic List the available magic commands / directives

In [2]:
#!fsharp

"Hello from F#" |> Console.WriteLine
let x = 10
let y = 15

Hello from F#


In [3]:
#!share --from fsharp x
x

10

In [4]:
#r "nuget:System.Device.Gpio,1.1.0-prerelease.20276.1"
#r "nuget:IoT.Device.Bindings,1.1.0-prerelease.20276.1"
#r "nuget:UnitsNet,4.68.0"

Installed package System.Device.Gpio version 1.1.0-prerelease.20276.1

Installed package UnitsNet version 4.68.0

Installed package IoT.Device.Bindings version 1.1.0-prerelease.20276.1

### Display the html

In [5]:
var pocketView = span(a[href:@"https://github.com/rondagdag/dotnetcode-jupyter-talk", target: "blank"](b("Presentation")));
display(pocketView)

Presentation

In [ ]:
var someHtml = HTML("<b style=\"color:blue\">Sensor data!</b>");
var tempOutput = display(someHtml);

In [ ]:
using Iot.Device.SenseHat;
using System;
var temperatures = new List<(int idx, double val)>();

async Task readTemperature(int count = 10){   
    using (var sh = new SenseHat())
    {
        var initcount = temperatures.Count;
        for (var i = initcount; i < initcount + count; i++){        
            temperatures.Add((i, sh.Temperature2.DegreesFahrenheit));
            tempOutput.Update(HTML($"<b style=\"color:green\">{sh.Temperature2.DegreesFahrenheit}</b>"));
            await Task.Delay(500);
        }
    }
}

### Read Temperatures

In [ ]:
await readTemperature(15);

In [ ]:
using XPlot.Plotly;

In [ ]:
var values = temperatures.Select(i => i.val);
values

In [ ]:
var openSeries = new Graph.Scatter
{
    name = "Open",
    x = temperatures.Select(i => i.idx),
    y = values
};


var chart = Chart.Plot(new[] {openSeries});
chart.WithTitle("Open");

### Display the chart

In [ ]:
display(chart)

### Instead of adding to list, let's setup Asycnronous Streams in C#

In [ ]:
async IAsyncEnumerable<double> ReadTemperatureAsync(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){
            await Task.Delay(1000);
            yield return (sh.Temperature2.DegreesFahrenheit);
        }
    };
}

In [ ]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;

In [ ]:
var output = display("Counting...");
PrimitiveDataFrameColumn<DateTime> dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes"); // Default length is 0.
PrimitiveDataFrameColumn<double> fahrenheit = new PrimitiveDataFrameColumn<double>("Fahrenheit"); // Makes a column of length 3. Filled with nulls initially

await foreach (var result in ReadTemperatureAsync(15))
{
   output.Update(result);
   dateTimes.Append(DateTime.Now);
   fahrenheit.Append(result);
}

### create a new Dataframe with 2 columns

In [ ]:
DataFrame df =  new DataFrame(dateTimes, fahrenheit);

In [ ]:
df

In [ ]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 5;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [ ]:
df

In [ ]:
var a = df.Rows.Take(2);
a

### Count the number of rows

In [ ]:
var total = df.Rows.Count();
total

In [ ]:
PrimitiveDataFrameColumn<double> celcius = new PrimitiveDataFrameColumn<double>("Celcius", total);
var newDf = df.Clone();
newDf.Columns.Insert(2, celcius);
newDf

In [ ]:
var info = newDf.Info();
info

In [ ]:
celcius.FillNulls(0, inPlace: true);
newDf

In [ ]:
DataFrameRow row0 = newDf.Rows[0];
row0

### Convert Fahrenheit and fill the Celcius

In [ ]:
newDf.Columns["Celcius"] = (fahrenheit - 32) * 5/9;
newDf

In [ ]:
var celciusMean = newDf.Columns["Celcius"].Mean();
celciusMean

In [ ]:
Chart.Plot(
    new Graph.Scatter()
    {
        x = newDf.Columns["DateTimes"],
        y = newDf.Columns["Celcius"]
    }
)

In [ ]:
var tempData = new List<double>();

async Task readTempData(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){        
            if(tempData.Count > 10){
                tempData.RemoveAt(0);
            }
            tempData.Add(sh.Temperature2.DegreesFahrenheit);
            Console.WriteLine(sh.Temperature2.DegreesFahrenheit);
            await Task.Delay(500);
        }
    }
}

In [ ]:
#!js 

if (typeof (notebookScope.interval) !== 'undefined') {
    clearInterval(notebookScope.interval);
}

notebookScope.plot = (sgvSelector, variableName) => {   
    let dtree_require = require.config({
        paths: {
            d3: "https://d3js.org/d3.v5.min"
        }
    });
    dtree_require(["d3"], function (d3) {       
        let svg = d3.select(sgvSelector);
        svg.selectAll("defs").remove();
        svg.selectAll("g").remove();
        
        var margin = {top: 0, right: 30, bottom: 30, left: 40},
            width = 260 - margin.left - margin.right,
            height = 200 - margin.top - margin.bottom;
        
        let container = d3
            .select(sgvSelector);
        
         var width = 260 
            scaleFactor = 20, 
            barHeight = 30;
         
         var graph = d3.select("body")
            .append("svg")
                .attr("width", width + margin.left + margin.right)
                .attr("height", height + margin.top + margin.bottom)
            .append("g")
                .attr("transform",
                      "translate(" + margin.left + "," + margin.top + ")");
        
        // Add X axis
          var x = d3.scaleLinear()
            .domain([0, 10])
            .range([ width, 0 ]);
        
          svg.append("g")
            .attr("transform", "translate(" + margin.left + "," + height + ")")
            .call(d3.axisBottom(x));

      // Add Y axis
          var y = d3.scaleLinear()
            .domain([101, 102])
            .range([ height, 0]);
        
          svg.append("g")
            .attr("transform", "translate(" + margin.left + ", 0)")
            .call(d3.axisLeft(y));

        updateD3Rendering = () => interactive.csharp.getVariable(variableName)
            .then(data => {
                console.log(data);
                container
                    .selectAll(".points")
                    .data(data, (d, i) => i)
                    .join(
                        enter => enter.append("circle")
                            .attr("cx", (d, i) => { console.log("i:" + i); return margin.left + x(i); } )
                            .attr("cy", function (d) { console.log("d:" + d); return y(d); } )
                            .attr("r", 2.5)
                            .style("fill", "#69b3a2")
                            .transition()
                            .duration(2000)                            
                            .attr("r", 0 ).remove(),
                        update => update
                            .transition()
                             .attr("class", "line")
                             .attr("d", valueline)
                            );
            });
        notebookScope.interval = setInterval(() => updateD3Rendering(), 1000);
    });
}

### Plot and call javascript 

In [ ]:
#!html
<svg id="dataPlot1" width=500 height=200>
</svg>

#!js
notebookScope.plot("svg#dataPlot1", "tempData")

In [ ]:
await readTempData(50);

In [ ]:
tempData.Count

In [ ]:
using System;
using System.Drawing;
async Task readTempCondition(int count = 10, double maxTemp = 100){   
    using (var ledMatrix = new SenseHatLedMatrixI2c())
    using (var sh = new SenseHat())
    {
        ledMatrix.Fill(Color.Black);
        for (var i = 0; i < count; i++){
            var temp = sh.Temperature2.DegreesFahrenheit;
            Console.WriteLine(temp);
            if (temp > maxTemp) {
                ledMatrix.Fill(Color.Red);
            } 
            await Task.Delay(500);
        }
    }
}

### read temperature 50 times and add max temp condition

In [ ]:
await readTempCondition(15, 101.7);

### clear the ledMatrix to "black"

In [ ]:
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}


## @rondagdag

## https://www.linkedin.com/in/rondagdag